# Exemple sur le modèle jouet Campbell

In [42]:
import numpy as np
import os
import gpflow

In [43]:
%reload_ext autoreload
%autoreload 2

import model_class

On récupère les entrées

In [44]:
hs_list = np.arange(1, 3.1, 0.5)
msl_list = np.arange(0, 1.1, 0.1)
print(hs_list)
x_train=[]
for hs in hs_list:
    for msl in msl_list:
        x_train.append([hs,msl])
x_train=np.array(x_train)
print("taille de x_train:",x_train.shape)

[1.  1.5 2.  2.5 3. ]
taille de x_train: (55, 2)


In [45]:
base_folder = "data_malo_GP"

y_train = []

for hs in hs_list:
    for msl in msl_list:

        folder_name = f"run_msl_{msl:.1f}_hs_{hs:.1f}"
        file_path = os.path.join(base_folder, folder_name, "scattered_map_final.bin")

        # Lecture du fichier binaire
        data = np.fromfile(file_path, dtype=np.float32)


        y_train.append(data)

y_train = np.array(y_train)
print("Taille de y_train :", y_train.shape)


Taille de y_train : (55, 2929740)


In [46]:
print("type:",y_train.dtype)
print("Présence de NaN:",np.isnan(y_train).any())
print("Présence d'infini:",np.isinf(y_train).any())
print("min =", np.nanmin(y_train))
print("max =", np.nanmax(y_train))
print("mean =", np.nanmean(y_train))
print("std =", np.nanstd(y_train))

type: float32
Présence de NaN: True
Présence d'infini: False
min = -3.4027969e+38
max = 3.402817e+38


c:\Users\Victor\anaconda3\envs\MachineLearning\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


mean = nan
std = nan


On a des valeurs extrèmes et des NaN :(

In [47]:
mask_extreme = np.abs(y_train) > 1e30
np.where(mask_extreme)
print("nb NaN       :", np.isnan(y_train).sum())
print("nb extrêmes  :", mask_extreme.sum())

nb NaN       : 258076
nb extrêmes  : 8730492


In [48]:
y_train_clean = np.where(mask_extreme, np.nan, y_train)
y_train_clean = np.nan_to_num(y_train_clean, nan=0.0)

mask_extreme = np.abs(y_train_clean) > 1e30
np.where(mask_extreme)
print("nb NaN       :", np.isnan(y_train_clean).sum())
print("nb extrêmes  :", mask_extreme.sum())

nb NaN       : 0
nb extrêmes  : 0


In [49]:
n_pc = 5  # On choisit arbitrairement le nombre de composantes principales (d'après la thèse, 5 semble un bon compromis)
theta = 3 # On choisit arbitrairement la valeur de theta 
sigma = 1 # et de sigma
param = [theta, sigma]

kernel = gpflow.kernels.SquaredExponential(lengthscales=param[0], variance=param[1]**2) + gpflow.kernels.White(variance=1e-6)

In [50]:
# ACP
ACP = model_class.ACP_classique(n_pc,[theta,sigma])
ACP.train(x_train,y_train_clean,kernel_fn=kernel)

c:\Users\Victor\anaconda3\envs\MachineLearning\Lib\site-packages\sklearn\decomposition\_pca.py:775: RuntimeWarning: overflow encountered in square
  self.explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\Victor\anaconda3\envs\MachineLearning\Lib\site-packages\sklearn\decomposition\_pca.py:786: RuntimeWarning: overflow encountered in square
  X_centered **= 2
c:\Users\Victor\anaconda3\envs\MachineLearning\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\Victor\anaconda3\envs\MachineLearning\Lib\site-packages\sklearn\decomposition\_pca.py:789: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var
c:\Users\Victor\anaconda3\envs\MachineLearning\Lib\site-packages\sklearn\decomposition\_pca.py:793: RuntimeWarning: invalid value encountered in scalar subtract
  self.noise_variance_ = total_var - xp.sum(sel


--- Entraînement du modèle GP pour la composante principale  1 ---
Modèle GP créé pour la composante principale  1 . Optimisation des hyperparamètres...
Optimisation terminée.

--- Entraînement du modèle GP pour la composante principale  2 ---
Modèle GP créé pour la composante principale  2 . Optimisation des hyperparamètres...
Optimisation terminée.

--- Entraînement du modèle GP pour la composante principale  3 ---
Modèle GP créé pour la composante principale  3 . Optimisation des hyperparamètres...
Optimisation terminée.

--- Entraînement du modèle GP pour la composante principale  4 ---
Modèle GP créé pour la composante principale  4 . Optimisation des hyperparamètres...
Optimisation terminée.

--- Entraînement du modèle GP pour la composante principale  5 ---
Modèle GP créé pour la composante principale  5 . Optimisation des hyperparamètres...
Optimisation terminée.
--- Analyse en Composantes Principales ---
Variance expliquée par les 5 premières composantes : [nan nan nan nan na

Bon bah ça à l'air de fonctionner, maintenant on peut voir ce que ca donne

In [51]:
Y_test_reconstruct_ACP = ACP.predict(x_train)  # On prédit sur les données d'entraînement pour vérifier la reconstruction
mse_ACP = np.mean((y_train_clean - Y_test_reconstruct_ACP)**2)
print("MSE ACP sur données d'entraînement :", mse_ACP)

Prédiction en cours...
MSE ACP sur données d'entraînement : 8.678455967099293e+56


Ah bah ca va pas du tout là !
Je ne sais pas à quoi c'est lié...